# Real footprinting data (simple case)

Rationale: After successfully evaluating the `footprint` module on artificial data, the module is to be tested on real data. To simplify evaluation of the success of this, I load a simplified version first, containing only the control # 7 (reference trace, without added ligand) and the other control traces (# 14 and 21) from MS010.

Realization:

- load all traces from HexA.csv into `trace_list`

In [1]:
import footprint
trace_list = footprint.get_data()

- fetch all traces with Ltot_conc = 0M

In [2]:
conc_0_traces = []
for t in trace_list:
    print(t.Ltot_conc,t.file_name)
    if t.Ltot_conc == 0:
        conc_0_traces.append(t)
#print(len(conc_0_traces))

3.0 103-23-16-6-33 PM.fsa
1.0 113-23-16-7-08 PM.fsa
0.3 123-23-16-7-42 PM.fsa
0.1 133-23-16-8-17 PM.fsa
0.0 143-23-16-8-51 PM.fsa
10.0 83-23-16-5-24 PM.fsa
5.0 93-23-16-5-58 PM.fsa
10.0 13-23-16-1-22 PM.fsa
5.0 23-23-16-1-56 PM.fsa
3.0 33-23-16-2-31 PM.fsa
1.0 43-23-16-3-05 PM.fsa
0.3 53-23-16-3-40 PM.fsa
0.1 63-23-16-4-15 PM.fsa
0.0 73-23-16-4-49 PM.fsa
10.0 153-23-16-9-26 PM.fsa
5.0 163-23-16-10-01 PM.fsa
3.0 173-23-16-10-35 PM.fsa
1.0 183-23-16-11-10 PM.fsa
0.3 193-23-16-11-44 PM.fsa
0.1 203-24-16-12-19 AM.fsa
0.0 213-24-16-12-54 AM.fsa


- set trace with most peaks as reference (arbitrary)

In [3]:
peaknr = [len(c.peaks) for c in conc_0_traces]

ref = [c for c in conc_0_traces if len(c.peaks) == max(peaknr)][0]

traces = [c for c in conc_0_traces if len(c.peaks) != max(peaknr)]

print(len(ref.peaks))
print(len(traces))
print(peaknr)

146
2
[139, 129, 146]


--> we have one `ref` with 146 peaks, and two other traces with less peaks...

In [4]:
footprint.cluster_peaks(ref,traces)

footprint.calculate_deviance_for_all_peaks(ref,traces)

#footprint.determine_factor_numerically(ref, traces[0])

#footprint.add_fractional_occupancies(ref,trace)

#footprint.calculate_free_ligand_concentration(ref,traces)

INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace 

83929.0

- check the clustered peaks

In [5]:
for i in footprint.give_all_clustered_peaks(ref,traces): 
    #print(i) ## extensive output
    #print("---")
    print(i[0].cluster,i[0].size_bp, i[0].peak_height,[n.peak_height for n in i[1]]) ##compact output


1 5.8792 55.0 []
2 9.745 165.0 [160.0, 107.0]
3 11.7584 1159.0 [370.0, 264.0]
4 14.094 529.0 [275.0, 156.0]
5 14.9799 794.0 [340.0, 218.0]
6 16.5906 555.0 [404.0, 307.0]
7 17.5571 7158.0 [5928.0, 4563.0]
8 19.4094 193.0 [94.0, 78.0]
9 20.9396 124.0 [69.0, 51.0]
10 23.1141 1968.0 [837.0, 602.0]
11 25.0433 330.0 [157.0, 98.0]
12 25.7443 221.0 [124.0, 62.0]
13 27.6595 2708.0 [1177.0, 799.0]
14 28.5934 1522.0 [674.0, 448.0]
15 30.265 183.0 [81.0, 52.0]
16 31.5906 887.0 [381.0, 246.0]
17 32.6324 376.0 [187.0, 115.0]
18 33.3824 1122.0 [514.0, 338.0]
19 34.289 905.0 [406.0, 280.0]
20 36.0466 171.0 [68.0, 51.0]
21 37.2852 75.0 []
22 38.6945 126.0 [63.0]
23 40.7605 962.0 [405.0, 278.0]
24 42.3746 3048.0 [1388.0, 920.0]
25 44.0932 291.0 [118.0, 79.0]
26 45.0034 1644.0 [679.0, 465.0]
27 46.8445 86.0 []
28 48.2865 4537.0 [2014.0, 1369.0]
29 49.7458 518.0 [251.0, 153.0]
30 50.5254 2118.0 [917.0, 632.0]
31 52.0116 65.0 []
32 52.8942 558.0 [224.0, 151.0]
33 55.0383 6943.0 [3020.0, 2182.0]
34 56.0333 

- find the optimal factors based on a single peak, and optimize the traces with this factor (!! NB: This is not necessarily the same peak with the current version of the `footprint` module!)

In [6]:
import imp
imp.reload(footprint)

zero = footprint.determine_factor_single_peak(ref, traces[0], 1, 1, True)
one = footprint.determine_factor_single_peak(ref, traces[1], 1,1, True)

INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace 

In [7]:
print(zero)
print(one)

2.249258160237389
3.1940298507462686


- correct the peaks with the determined factors

In [8]:
footprint.correct_peaks_with_factor(traces[0],zero)
footprint.correct_peaks_with_factor(traces[1],one)


- check the clustered peaks again

In [9]:
for i in footprint.give_all_clustered_peaks(ref,traces): 
    #print(i) ## extensive output
    #print("---")
    print(i[0].cluster,i[0].peak_height,[n.peak_height for n in i[1]], [i[0].peak_height-n.peak_height for n in i[1]]) ##compact output


1 55.0 [] []
2 165.0 [359.8813056379822, 341.76119402985074] [-194.88130563798222, -176.76119402985074]
3 1159.0 [832.2255192878339, 843.2238805970148] [326.7744807121661, 315.77611940298516]
4 529.0 [618.545994065282, 498.2686567164179] [-89.545994065282, 30.731343283582078]
5 794.0 [764.7477744807122, 696.2985074626865] [29.252225519287776, 97.70149253731347]
6 555.0 [908.7002967359051, 980.5671641791045] [-353.7002967359051, -425.56716417910445]
7 7158.0 [13333.60237388724, 14574.358208955224] [-6175.602373887241, -7416.358208955224]
8 193.0 [211.43026706231456, 249.13432835820896] [-18.430267062314556, -56.13432835820896]
9 124.0 [155.19881305637983, 162.8955223880597] [-31.198813056379834, -38.8955223880597]
10 1968.0 [1882.6290801186944, 1922.8059701492537] [85.37091988130555, 45.19402985074635]
11 330.0 [353.13353115727006, 313.0149253731343] [-23.133531157270056, 16.985074626865696]
12 221.0 [278.9080118694362, 198.02985074626866] [-57.90801186943622, 22.970149253731336]
13 270

In [22]:
rmsd_zero = footprint.calculate_deviance_for_all_peaks(ref,traces[0],1,1,True)
rmsd_one = footprint.calculate_deviance_for_all_peaks(ref,traces[0],1,1,False)

INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace contains peaks which are not included in deviation calculation.
INFO: Trace 

In [23]:
print(rmsd_zero,rmsd_one)

17.865209345585946 40224.077151335325


In [32]:
integrated_peak_height = sum([ref_peak.peak_height for ref_peak in ref.peaks])
#print([ref_peak.peak_height for ref_peak in ref.peaks])
print(integrated_peak_height)

171233.0
